# M.net 에서 데이터를 꺼내보자!

* 실시간 인기 순위를 가져오고 저장도 수행
* 매일 오후 12시를 기준으로 데이터를 불러옴
* 저장 파일 형식 : CSV / 저장 위치 : D:\Download 라고 임의 설정

In [ ]:
## 패키지 불러오기

import requests
import bs4
import csv
import time
import os

## 순위표 내 앨범 사진을 저장하기 위한 클래스 작성

def save_img(img_url, rank, date) :
    
        response = requests.get(img_url)
        content = response.content
        
        filename = 'D:\Download\{}\\{:04}.jpeg'.format(date, rank)
        file = open(filename, 'wb')
        file.write(content)

## 언제, 몇 개나 순위를 추출할 거냐?        
 
date = int(input ('원하시는 일자를 입력해 주세요. [ex : 20190525] : '))
hour = '12' # <-- 12시 데이터를 기준으로 추출
pages = int(input ('몇 페이지를 출력하고 싶나요? : '))

## 저장할 디렉토리 만들기

os.system('md D:\Download\{}'.format(date))

## csv, jpeg 파일로 저장하기

charts = [] # <-- 차트를 저장할 2차원 목록 초기화 시키기

print ('\n---- 결과 입니다. {}초 동안 기다려 주세요. 호출 중 입니다. ---'.format(pages))

for page in range(1, pages + 1) :
    
    ## 순위, 곡명, 가수 이름 추출하기
    
    url = ('http://www.mnet.com/chart/TOP100/{}{}?pNum={}'.format(date, hour, page))
    response = requests.get(url)
    text = response.text
    html = bs4.BeautifulSoup(text, 'html.parser')
    
    tr_list = html.find('table').find('tbody').find_all('tr') # <-- m.net 사이트 내 위의 정보 있는 곳 지정하기
 
    for tr in tr_list : # <-- 세부 데이터 추출하기

        rank = int ( tr.find('td', {'class':'MMLItemRank'}).find('span').text.rstrip("위") )
        title = tr.find('a', {'class':'MMLI_Song'}).text
    
        img_url = tr.find('img')['src'] # <-- 이미지 저장 위치 지정
        save_img(img_url, rank, date) # <-- 미리 만들어 놓은 클래스 이용해서 이미지 파일 저장하기
        
        ## 에러 코드를 위한 대비용 코드
        ## 발생하는 에러 : 저작권이 환수되어 더 이상 플레이가 안되는 곡이 순위 리스트 상에 있을 때 데이터 추출이 멈추는 에러 대비용
        
        try : # <-- 저작권 있는 곡의 데이터 위치 지정용
            
            artist = tr.find('a', {'class':'MMLIInfo_Artist'}).text
            album = tr.find('a', {'class':'MMLIInfo_Album'}).text
        
        except : # <-- 저작권 없는 곡의 데이터 위치 지정용
            
            art_alb = tr.find('div', {'class':'MMLITitle_Info'}).text.split('/')[0]
            artist = art_alb[0].strip()
            album = art_alb[1].strip()
        
        ## 추출된 데이터를 chart 2차원 목록에 저장
        
        charts.append( [rank, title, artist, album] )
    
    ## DoS 공격 오인을 피하기 위한 방어용 코드 추가
    
    time.sleep(1) # <-- 순위를 찾을 때 Query 문에서 1초씩 지연시간을 추가함

## 파일 저장을 위한 단계
    
for chart in charts : # <-- 리스트 확인용 코드
    print (chart)
        
answer = input ('\n위의 순위를 Excel 파일로 저장 하시겠어요? : ')

if answer.lower() in ['yes', 'ye', 'y'] : # <-- 답변 처리용 반복문
    
    filename = 'D:\Download\{0:}\charts_{0:}.csv'.format(date)
    file = open(filename, 'w', newline = '', encoding = 'utf-8-sig')
    csvfile = csv.writer(file)
    
    for chart in charts : 
        
        print (chart)

    csvfile.writerows(charts)
        
file.close()